In [3]:
from music21 import converter, instrument, note, chord
import sys
import numpy as np
import os
from imageio import imwrite

def extractNote(element):
    return int(element.pitch.ps)

def extractDuration(element):
    return element.duration.quarterLength

def get_notes(notes_to_parse):

    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    durations = []
    notes = []
    start = []

    for element in notes_to_parse:
        if isinstance(element, note.Note):
            if element.isRest:
                continue

            start.append(element.offset)
            notes.append(extractNote(element))
            durations.append(extractDuration(element))
                
        elif isinstance(element, chord.Chord):
            if element.isRest:
                continue
            for chord_note in element.notes:
                start.append(element.offset)
                durations.append(extractDuration(element))
                notes.append(extractNote(chord_note))

    return {"start":start, "pitch":notes, "dur":durations}


def midi2image(midi_path, max_repetitions = float("inf"), resolution = 0.25, lowerBoundNote = 21, upperBoundNote = 127, maxSongLength = 100):
    #print(midi_path)
    try:
        mid = converter.parse(midi_path)

        instruments = instrument.partitionByInstrument(mid)

        data = {}

        try:
            i=0
            for instrument_i in instruments.parts:
                notes_to_parse = instrument_i.recurse()

                notes_data = get_notes(notes_to_parse)
                if len(notes_data["start"]) == 0:
                    continue

                if instrument_i.partName is None:
                    data["instrument_{}".format(i)] = notes_data
                    i+=1
                else:
                    data[instrument_i.partName] = notes_data

        except:
            notes_to_parse = mid.flat.notes
            data["instrument_0"] = get_notes(notes_to_parse)

        for instrument_name, values in data.items():
            # https://en.wikipedia.org/wiki/Scientific_pitch_notation#Similar_systems

            pitches = values["pitch"]
            durs = values["dur"]
            starts = values["start"]

            index = 0
            while index < max_repetitions:
                matrix = np.zeros((upperBoundNote-lowerBoundNote,maxSongLength))


                for dur, start, pitch in zip(durs, starts, pitches):
                    dur = int(dur/resolution)
                    start = int(start/resolution)

                    if not start > index*(maxSongLength+1) or not dur+start < index*maxSongLength:
                        for j in range(start,start+dur):
                            if j - index*maxSongLength >= 0 and j - index*maxSongLength < maxSongLength:
                                matrix[pitch-lowerBoundNote,j - index*maxSongLength] = 255

                if matrix.any(): # If matrix contains no notes (only zeros) don't save it
                    imwrite(midi_path.split("/")[-1].replace(".mid",f"_{instrument_name}_{index}.png"),matrix.astype(np.uint8))
                    index += 1
                else:
                    break
        #print("worked")
        return True
    except:
        #print("Dont work")
        return False

In [4]:
def start_generating_images(path):
    list_paths = []
    for subdir, dirs, files in os.walk(path):
        for file in files:
            if file.endswith("mid"):
                #print(os.path.join(subdir, file))
                list_paths.append(os.path.join(subdir, file))
                break;
    worked =0
    didnt = 0;
    for path in list_paths[2688:]:
        print(path)
        if midi2image(path, 1):
            worked +=1
        else:
            didnt +=1
    print("didnt", didnt)
    print("worked", worked)
        

In [5]:
                
if __name__ == "__main__":
    midi_path = "C:/soft/bb/MusicGenerator/midi"
    start_generating_images(midi_path)
    #midi2image("C:\soft\bb\MusicGenerator\midi\ hack  Mutation\DotHack-Innis.mid", 1)
    #
#         midi2image(midi_path, 1)
#     else:
#         midi2image(midi_path)

C:/soft/bb/MusicGenerator/midi\Thunder Force II\Sega_Techno-Soft_1989_Thunder-Force-II_07-03-05_Track-01.mid
C:/soft/bb/MusicGenerator/midi\Thunder Force III\base.mid


C:\Users\rajta\anaconda3\lib\site-packages\music21\midi\translate.py:790: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent PROGRAM_CHANGE, track=1, channel=10, data=0>; getting generic UnpitchedPercussion
  warnings.warn(


C:/soft/bb/MusicGenerator/midi\Thunder Force V\tf5188.mid


C:\Users\rajta\anaconda3\lib\site-packages\music21\midi\translate.py:790: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent PROGRAM_CHANGE, track=10, channel=10, data=16>; getting generic UnpitchedPercussion
  warnings.warn(


C:/soft/bb/MusicGenerator/midi\Thunder Spirits\Tshiscre.mid


...

C:/soft/bb/MusicGenerator/midi\Yu Yu Hakusho\YuYuOP.mid


C:\Users\rajta\anaconda3\lib\site-packages\music21\midi\translate.py:785: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=0, channel=None, data=b'\x94\xf7\x8f\xce\x82\xdd\x82\xcc\x94\x9a\x92e(\x97H\x97V\x94\x92\x8f\x91\x83I\x81[\x83v\x83j\x83\x93\x83O\x83e\x81[\x83}) [\x94n\x93n\x8f\xbc\x8eq] '>; getting generic Instrument
  warnings.warn(
C:\Users\rajta\anaconda3\lib\site-packages\music21\midi\translate.py:790: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent PROGRAM_CHANGE, track=11, channel=10, data=8>; getting generic UnpitchedPercussion
  warnings.warn(


C:/soft/bb/MusicGenerator/midi\Yu Yu Hakusho_ Makyo Toitsusen (Japan)\Genesis__Yu_Yu_Hakusho_Makyo_Toitsusen__Jin.mid
C:/soft/bb/MusicGenerator/midi\Yu-Gi-Oh! Forbidden Memories\meadow_shrine.mid
C:/soft/bb/MusicGenerator/midi\Yu-Gi-Oh! Ultimate Masters_ World Championship Tournament 2006\Free_Duel_-_Tier_2.mid
C:/soft/bb/MusicGenerator/midi\Yu-Gi-Oh! _ Worldwide Edition\YOWWE_KCCUP.mid
C:/soft/bb/MusicGenerator/midi\Yuu Yuu Hakusho (Japan)\yuu_yuu_hakusho_japan_-_arena_battle_theme.mid
C:/soft/bb/MusicGenerator/midi\Yuu Yuu Hakusho 2_ Kakutou no Sho (Japan)\yuu_yuu_hakusho_kakutou_no_syo_japan_-_credits.mid
C:/soft/bb/MusicGenerator/midi\Yuu Yuu Hakusho Final\YuuYuuFinalKuramaThemeSNES.mid
C:/soft/bb/MusicGenerator/midi\Yuuyuuki\Yuuyuuki12.mid
C:/soft/bb/MusicGenerator/midi\Zanac\zanac1a.mid
C:/soft/bb/MusicGenerator/midi\Zany Golf\PinballPalace.mid
C:/soft/bb/MusicGenerator/midi\Zelda\zcdiboss.mid
C:/soft/bb/MusicGenerator/midi\Zelda II_ The Adventure of Link\Alternate_Battle.mid
C:/

C:\Users\rajta\anaconda3\lib\site-packages\music21\midi\translate.py:790: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent PROGRAM_CHANGE, track=11, channel=10, data=16>; getting generic UnpitchedPercussion
  warnings.warn(


C:/soft/bb/MusicGenerator/midi\Zombies Ate My Neighbors\PoF.mid
C:/soft/bb/MusicGenerator/midi\Zone 66\Zone66_PlantationCrash-FM.mid
C:/soft/bb/MusicGenerator/midi\Zool\Zool-Theme.mid
C:/soft/bb/MusicGenerator/midi\Zoop\Tun3.mid
C:/soft/bb/MusicGenerator/midi\Zorro\zorro.mid
didnt 101
worked 245
